In [1]:
import os 
import torch 
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary
from torchvision.transforms import transforms
import matplotlib
import matplotlib.pyplot as plt

In [2]:
img_transform = transforms.Compose([
    transforms.ToTensor(),    
])

In [ ]:
train_data = datasets.CIFAR10(  # Loads the CIFAR-10 training dataset
    root="./data/train",  # Directory where the dataset is stored
    train=True,  # Specifies that we want the training set
    download=True,  # Downloads the dataset if it is not already available
    transform=img_transform  # Applies the defined transformations to the dataset
)

test_data = datasets.CIFAR10(  # Loads the CIFAR-10 test dataset
    root="./data/test",  # Directory where the dataset is stored
    train=False,  # Specifies that we want the test set
    download=True,  # Downloads the dataset if it is not already available
    transform=img_transform  # Applies the same transformations as for the training set
)

In [4]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)


In [ ]:
(next(iter(train_loader)))[0].shape

In [6]:
image, label = next(iter(train_loader))

In [ ]:
plt.imshow(image[0].permute(1, 2, 0))

In [ ]:
from gan_model import ConditionalGenerator

generator = ConditionalGenerator()

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create generator
generator = ConditionalGenerator(latent_dim=100, num_classes=10)

# Load checkpoint if available (otherwise will use untrained weights)
checkpoint_path = 'checkpoints/cgan_generator_epoch100.pth'  # Change to your path
generator.load_state_dict(torch.load(checkpoint_path, map_location=device))

generator.to(device)
generator.eval()

# CIFAR-10 class names
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                 'dog', 'frog', 'horse', 'ship', 'truck']

# Choose a class (0-9)
class_idx = 5  # Generate a cat (change to any number 0-9)
print(f"Generating a {cifar_classes[class_idx]}")

# Generate one image
with torch.no_grad():
    # Create random noise
    z = torch.randn(1, 100, device=device)
    
    # Create class label
    label = torch.tensor([class_idx], device=device)
    
    # Generate image
    fake_image = generator(z, label)
    
    # Convert from [-1,1] to [0,1] range
    fake_image = fake_image * 0.5 + 0.5

# Display image
plt.figure(figsize=(3, 3))
plt.imshow(fake_image[0].cpu().permute(1, 2, 0))
plt.title(f"Generated {cifar_classes[class_idx]}")
plt.axis('off')
plt.show()